In [ ]:
import numpy as np
import pandas as pd
import datetime as dt
from sqlalchemy import create_engine
import urllib
import pyodbc
import requests
from time import sleep

In [ ]:
pd.options.display.min_rows = 100

In [ ]:
# Download from https://health.mo.gov/safety/medical-marijuana/licensed-facilities.php

In [ ]:
today_date = dt.date.today()
# today_date = dt.date(2022, 2, 16)

today_year = today_date.strftime("%Y")
today_month = today_date.strftime("%m")
today_day = today_date.strftime("%d")

### Create SQL Connection

In [ ]:
# This creates the connection to the California Cannabis files in SQL
params = urllib.parse.quote_plus("DRIVER={ODBC Driver 17 for SQL Server};"
                                 "SERVER=bespoke-database-1.cmevrozrcs7c.us-west-2.rds.amazonaws.com;"
                                 "DATABASE=mo_cannabis;"
                                 "UID=admin;"
                                 "PWD=N19lrqxnurTUJLJT6GFe")
engine_con = create_engine("mssql+pyodbc:///?odbc_connect={}".format(params))

### Read in the Data

In [ ]:
cultivation_df = pd.read_excel(f'licenses/licensed_cultivation_{today_year}_{today_month}_{today_day}.xlsx', header=1)

In [ ]:
dispensary_df = pd.read_excel(f'licenses/licensed_dispensary_{today_year}_{today_month}_{today_day}.xlsx', header=1)

In [ ]:
manufacturing_df = pd.read_excel(f'licenses/licensed_infused_product_manufacturing_{today_year}_{today_month}_{today_day}.xlsx', header=1)

In [ ]:
laboratory_df = pd.read_excel(f'licenses/licensed_laboratory_testing_{today_year}_{today_month}_{today_day}.xlsx', header=1)

In [ ]:
transportation_df = pd.read_excel(f'licenses/licensed_transportation_{today_year}_{today_month}_{today_day}.xlsx', header=1)

In [ ]:
cultivation_df['license_description'] = 'Cultivation'

In [ ]:
dispensary_df['license_description'] = 'Dispensary'

In [ ]:
manufacturing_df['license_description'] = 'Manufacturing'

In [ ]:
laboratory_df['license_description'] = 'Laboratory'

In [ ]:
transportation_df['license_description'] = 'Laboratory'

In [ ]:
manufacturing_df = manufacturing_df.rename(columns={' Contact \nInformation 1':'Contact \nInformation 1'})

In [ ]:
all_licenses_df = cultivation_df.append(dispensary_df).append(manufacturing_df).append(laboratory_df).append(transportation_df)

In [ ]:
all_licenses_df = all_licenses_df.sort_values(by='Entity Name')
all_licenses_df = all_licenses_df.reset_index().drop(columns='index')

### Data Cleaning

In [ ]:
all_licenses_df = all_licenses_df.rename(columns={
    'Approved to Operate':'license_approved',
    'License \nNumber':'license_number',
    'Entity Name':'name_legal',
    'City':'contact_city',
    'State':'contact_state',
    'Postal Code':'contact_zip',
    'Contact \nInformation 1':'contact_owner_1',
    'Contact \nInformation 2':'contact_owner_2',
    'Contact \nPhone':'contact_phone'
})

In [ ]:
all_licenses_df['license_approved'] = all_licenses_df['license_approved'].fillna(0).replace('ü', 1)

In [ ]:
phone_list = []
for num in all_licenses_df['contact_phone']:
    num = str(num).replace(' ', '').replace('(', '').replace(')', '').replace('-', '')
    phone_list.append(num)
all_licenses_df['contact_phone'] = phone_list

In [ ]:
all_licenses_df['company_roll_up'] = ''
all_licenses_df['roll_up_id'] = ''

In [ ]:
all_licenses_df = all_licenses_df[['license_number', 'license_description', 'name_legal', 'roll_up_id', 'company_roll_up', 'contact_city', 'contact_state', 'contact_zip', 'contact_owner_1', 'contact_owner_2', 'contact_phone', 'license_approved']]
all_licenses_df.head()

### Add Data from SQL

In [ ]:
old_sql_roll = pd.read_sql('mo_roll', con=engine_con)

In [ ]:
all_licenses_df = all_licenses_df.drop(columns='roll_up_id').drop(columns='company_roll_up')

In [ ]:
all_licenses_df = all_licenses_df.merge(old_sql_roll, how='left', on='license_number')

In [ ]:
all_licenses_df = all_licenses_df[['license_number', 'license_description', 'name_legal', 'roll_up_id', 'company_roll_up', 
                                   'contact_city', 'contact_state', 'contact_zip', 'contact_owner_1', 'contact_owner_2',
                                   'contact_phone', 'license_approved']]

all_licenses_df['is_current'] = 1

In [ ]:
all_licenses_df.head()

In [ ]:
all_licenses_df = all_licenses_df[all_licenses_df['license_number'].notnull()]

### Send to CSV to add Roll Up Info

In [ ]:
all_licenses_df.to_csv(f'edited_files/df_to_edit_{today_year}_{today_month}_{today_day}.csv', index=False)

In [ ]:
if sum(pd.read_csv(f'edited_files/df_to_edit_{today_year}_{today_month}_{today_day}.csv')['company_roll_up'].isnull()) == 0:
    df_comp_roll_added = pd.read_csv(f'edited_files/df_to_edit_{today_year}_{today_month}_{today_day}.csv')
else:
    print(error)

### Add roll_up_ids

In [ ]:
df_comp_roll_added['roll_up_id'] = df_comp_roll_added['roll_up_id'].fillna('')

In [ ]:
df_with_roll = df_comp_roll_added[df_comp_roll_added['roll_up_id'] != '']

In [ ]:
df_without_roll = df_comp_roll_added[df_comp_roll_added['roll_up_id'] == '']

In [ ]:
new_comp_rolls = list(df_without_roll['company_roll_up'].unique())

In [ ]:
new_roll_id = []

In [ ]:
max_roll = int(str(list(df_with_roll.sort_values(by='roll_up_id')['roll_up_id'])[-1])[-5:])

In [ ]:
for n in new_comp_rolls:
    max_roll += 1
    new_roll_id.append(max_roll)

In [ ]:
for n in range(len(new_comp_rolls)):
    location_list = list(df_without_roll[df_comp_roll_added['company_roll_up'] == new_comp_rolls[n]].index)
    for x in location_list:
        df_without_roll['roll_up_id'][x] = new_roll_id[n]

In [ ]:
list(df_without_roll.index)

In [ ]:
new_roll_id_with_mo = []
for n in list(df_without_roll.index):
    roll_string = str(df_without_roll['roll_up_id'][n])
    while len(roll_string) < 5:
        roll_string = '0' + roll_string
    roll_string = 'MO-' + roll_string
    new_roll_id_with_mo.append(roll_string)
df_without_roll['roll_up_id'] = new_roll_id_with_mo

In [ ]:
df_without_roll

In [ ]:
df_total = df_with_roll.append(df_without_roll).sort_index()

In [ ]:
df_total

### Compare to SQL

In [ ]:
old_sql_main = pd.read_sql('mo_main', con=engine_con)

In [ ]:
new_lic_list = []
no_longer_current_list = []

for lic in list(df_total['license_number']):
    if lic not in list(old_sql_main['license_number']):
        new_lic_list.append(lic)
        
for lic in list(old_sql_main['license_number']):
    if lic not in list(df_total['license_number']):
        no_longer_current_list.append(lic)

In [ ]:
df_for_sql = df_total[df_total['license_number'].isin(new_lic_list)]

In [ ]:
df_for_sql

In [ ]:
df_not_current = old_sql_main[old_sql_main['license_number'].isin(no_longer_current_list)]

In [ ]:
df_not_current['is_current'] = 0

In [ ]:
df_with_not_current = df_total.append(df_not_current)[['license_number', 'license_description', 'name_legal', 'license_approved', 'is_current']]

In [ ]:
df_with_not_current['is_current'] = df_with_not_current['is_current'].fillna(1).astype(int)

In [ ]:
df_with_not_current.head()

### Seperate for SQL

In [ ]:
mo_main_for_sql = df_for_sql[['license_number', 'name_legal', 'license_description', 'license_approved', 'is_current']]

In [ ]:
mo_roll_for_sql = df_for_sql[['license_number', 'roll_up_id', 'company_roll_up']]

In [ ]:
mo_contact_for_sql = df_for_sql[['license_number', 'contact_owner_1', 'contact_owner_2', 'contact_phone', 'contact_city', 'contact_zip', 'contact_state']]

In [ ]:
mo_main_for_sql.to_sql('mo_main', con=engine_con, index=False, if_exists='append')

In [ ]:
mo_roll_for_sql.to_sql('mo_roll', con=engine_con, index=False, if_exists='append')

In [ ]:
mo_contact_for_sql.to_sql('mo_contact', con=engine_con, index=False, if_exists='append')

### Creating and Running main_alter

In [ ]:
mo_main_sql_for_alter = pd.read_sql('mo_main', con=engine_con)

In [ ]:
to_alter_list = []
no_longer_active = []

for lic in list(df_with_not_current['license_number'].unique()):
    if list(mo_main_sql_for_alter[mo_main_sql_for_alter['license_number'] == lic]['is_current'])[0] != list(df_with_not_current[df_with_not_current['license_number'] == lic]['is_current'])[0]:
        print(lic, 'current')
        no_longer_active.append(lic)
        to_alter_list.append(lic)
    elif list(mo_main_sql_for_alter[mo_main_sql_for_alter['license_number'] == lic]['license_approved'])[0] != list(df_with_not_current[df_with_not_current['license_number'] == lic]['license_approved'])[0]:
        print(lic, 'approved')
        to_alter_list.append(lic)
    elif list(mo_main_sql_for_alter[mo_main_sql_for_alter['license_number'] == lic]['license_description'])[0] != list(df_with_not_current[df_with_not_current['license_number'] == lic]['license_description'])[0]:
        print(lic, 'type')
        to_alter_list.append(lic)

In [ ]:
mo_main_alter = df_with_not_current[df_with_not_current['license_number'].isin(to_alter_list)]

In [ ]:
mo_main_alter = mo_main_alter.reset_index().drop(columns='index')

In [ ]:
mo_main_alter['date_no_longer_active'] = ''
mo_main_alter['date_no_longer_active'] = pd.to_datetime(mo_main_alter['date_no_longer_active'])

for n in range(len(mo_main_alter)):
    if mo_main_alter['license_number'][n] in no_longer_active:
        mo_main_alter['date_no_longer_active'][n] = today_date

In [ ]:
mo_main_alter

In [ ]:
mo_main_alter.to_sql('mo_main_alter', con=engine_con, index=False, if_exists='replace')

In [ ]:
# This code runs the query noramally run in SQL to update mo_main using mo_main_alter

mo_conn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};'
                      'SERVER=bespoke-database-1.cmevrozrcs7c.us-west-2.rds.amazonaws.com;'
                      'DATABASE=mo_cannabis;'
                      'UID=admin;'
                      'PWD=N19lrqxnurTUJLJT6GFe;')

mo_cursor = mo_conn.cursor()

mo_cursor.execute('SELECT * FROM mo_cannabis.dbo.mo_main')

mo_cursor.execute('''
Update t1
SET t1.[license_description] = t2.[license_description],
    t1.[license_approved] = t2.[license_approved],
    t1.[is_current] = t2.[is_current]
    
FROM [mo_cannabis].[dbo].[mo_main] as t1
INNER JOIN [mo_cannabis].[dbo].[mo_main_alter] as t2
ON t1.[license_number] = t2.[license_number];
                  ''')

mo_conn.commit()

### all_main

In [ ]:
mo_main_for_all_main = pd.read_sql('mo_main', con=engine_con)
mo_roll_for_all_main = pd.read_sql('mo_roll', con=engine_con)

In [ ]:
mo_for_all_main = mo_main_for_all_main.merge(mo_roll_for_all_main, on='license_number')

In [ ]:
mo_for_all_main

In [ ]:
# This creates the connection to the all_main file in SQL

all_cannabis_params = urllib.parse.quote_plus("DRIVER={ODBC Driver 17 for SQL Server};"
                                 "SERVER=bespoke-database-1.cmevrozrcs7c.us-west-2.rds.amazonaws.com;"
                                 "DATABASE=all_cannabis;"
                                 "UID=admin;"
                                 "PWD=N19lrqxnurTUJLJT6GFe")
all_cannabis_con = create_engine("mssql+pyodbc:///?odbc_connect={}".format(all_cannabis_params))

In [ ]:
# This reads in the all_main data and narrows it down to the relevent data
all_main_sql = pd.read_sql('all_main', con=all_cannabis_con)
all_main_sql_mo = all_main_sql[all_main_sql['us_state'] == 'MO']
all_main_sql_mo = all_main_sql_mo.reset_index().drop(columns='index')

In [ ]:
mo_for_all_main['license_status'] = mo_for_all_main['license_approved'].replace({1: 'Active', 0:'Inactive'})
mo_for_all_main['us_state'] = 'MO'
mo_for_all_main['license_category'] = mo_for_all_main['license_description']
mo_for_all_main['expiration_date'] = None

In [ ]:
mo_for_all_main = mo_for_all_main[['license_number', 'name_legal', 'roll_up_id', 'license_status', 'is_current', 'license_description', 'us_state', 'license_category', 'expiration_date']]
mo_for_all_main['license_category'].value_counts()

In [ ]:
mo_for_all_main['license_category'] = mo_for_all_main['license_category'].replace({'Dispensary': 'Retailer', 'Cultivation': 'Cultivator', 'Manufacturing': 'Manufacturer'})

In [ ]:
mo_for_all_main_cleaned = mo_for_all_main.rename(columns={'roll_up_id': 'rollup_id', 'name_legal': 'legal_name'})

In [ ]:
# This prevents duplicates
mo_for_all_main_cleaned = mo_for_all_main_cleaned[~mo_for_all_main_cleaned['license_number'].isin(all_main_sql_mo['license_number'].unique())]

In [ ]:
mo_for_all_main_cleaned

In [ ]:
mo_for_all_main_cleaned.to_sql('all_main', con=all_cannabis_con, index=False, if_exists='append')

### Alter all_main

In [ ]:
all_main_sql = pd.read_sql('all_main', con=all_cannabis_con)
all_main_sql_mo = all_main_sql[all_main_sql['us_state'] == 'MO']
all_main_sql_mo = all_main_sql_mo.reset_index().drop(columns='index')

In [ ]:
for lic in list(all_main_sql_mo['license_number'].unique()):
    if lic not in list(mo_for_all_main['license_number'].unique()):
        print(lic)

In [ ]:
all_main_sql_mo.shape

In [ ]:
all_main_sql_mo.head()

In [ ]:
to_alter_list = []

for lic in list(mo_for_all_main['license_number'].unique()):
    if list(all_main_sql_mo[all_main_sql_mo['license_number'] == lic]['is_current'])[0] != list(mo_for_all_main[mo_for_all_main['license_number'] == lic]['is_current'])[0]:
        print(lic, 'current')
        to_alter_list.append(lic)
#     elif list(all_main_sql_mo[all_main_sql_mo[''] == lic]['license_approved'])[0] != list(df_with_not_current[df_with_not_current['license_number'] == lic]['license_approved'])[0]:
#         print(lic, 'approved')
#         to_alter_list.append(lic)
    elif list(all_main_sql_mo[all_main_sql_mo['license_number'] == lic]['license_description'])[0] != list(mo_for_all_main[mo_for_all_main['license_number'] == lic]['license_description'])[0]:
        print(lic, 'type')
        to_alter_list.append(lic)

In [ ]:
mo_for_alter = mo_for_all_main[mo_for_all_main['license_number'].isin(to_alter_list)]

In [ ]:
mo_for_alter

In [ ]:
mo_for_alter.to_sql('mo_to_alter', con=all_cannabis_con, if_exists='replace', index=False)

In [ ]:
# This code runs the query noramally run in SQL to update ca_main using ca_main_alter

all_conn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};'
                      'SERVER=bespoke-database-1.cmevrozrcs7c.us-west-2.rds.amazonaws.com;'
                      'DATABASE=all_cannabis;'
                      'UID=admin;'
                      'PWD=N19lrqxnurTUJLJT6GFe;')

all_cursor = all_conn.cursor()

all_cursor.execute('SELECT * FROM all_cannabis.dbo.all_main')

all_cursor.execute('''
Update t1
SET t1.license_description = t2.license_description,
    t1.license_category = t2.license_category,
    t1.is_current = t2.is_current
    
FROM all_cannabis.dbo.all_main as t1
INNER JOIN all_cannabis.dbo.mo_to_alter as t2
ON t1.license_number = t2.license_number;
                  ''')

all_conn.commit()

### Bespoke Platform

In [ ]:
new_main = mo_for_all_main_cleaned.append(mo_for_alter)

In [ ]:
new_main

In [ ]:
new_main.shape

In [ ]:
new_main['expiration_date'] = new_main['expiration_date'].replace({None: np.nan})

In [ ]:
new_main = new_main.reset_index().drop(columns='index')

In [ ]:
new_main

In [ ]:
print(stop)

In [ ]:
all_main_sql_mo[all_main_sql_mo['license_number'] == 'CUL000050']

In [ ]:
new_main = all_main_sql_mo[all_main_sql_mo['license_number'].isin(['MAN000105', 'CUL000050', 'DIS000207', 'CUL000065', 'DIS000208', 'MAN000106', 'DIS000206', 'TRA000041', 'TRA000039', 'TRA000038'])]

In [ ]:
new_main = new_main.reset_index().drop(columns='index')

In [ ]:
def send_to_platform(data, production=bool):
    if production == 1:
        requests.post(
        url='https://bespoke-api-server-production.herokuapp.com/api/v1/licenses/bulk_update_licenses',
        json={'company_licenses': data},
        headers={'x-api-key': 'ASYNC-SERVER-API-KEY-6Lfy84qn9ew93ZwMXSYhdwqj'})
    elif production == 0:
        requests.post(
        url='https://bespoke-api-server-staging.herokuapp.com/api/v1/licenses/bulk_update_licenses',
        json={'company_licenses': data},
        headers={'x-api-key': '80030179832729980933'})
    

n = 0

while len(new_main) - n > 0:
    
    list_for_platform = []
    
    while len(list_for_platform) < 25 and n < len(new_main):
    
        if new_main['is_current'].iloc[n] == 1:
            is_current = True
        else:
            is_current = False

        list_for_platform.append({
        'license_number': new_main['license_number'][n],
        'rollup_id': new_main['rollup_id'][n],
        'legal_name': new_main['legal_name'][n],
        'license_status': new_main['license_status'][n],
        'is_current': is_current,
        'license_description': new_main['license_description'][n],
        'us_state': 'MO',
        'expiration_date': None,
        'license_category': new_main['license_category'][n]
        })
        
        n+=1
        
    send_to_platform(list_for_platform, production=1)
    
    print(n)
    
    sleep(3)

In [ ]:
new_main[new_main['is_current'] == 0]

In [ ]:
requests.post(
#             'https://bespoke-api-server-staging.herokuapp.com/api/v1/licenses/bulk_update_licenses',
            'https://bespoke-api-server-production.herokuapp.com/api/v1/licenses/bulk_update_licenses',
            json= {
          'company_licenses': [
              
              {
            'license_number': 'DIS000091',
            'rollup_id': 'MO-00013',
            'legal_name': 'Astro Farms Gamma LLC',
            'license_status': 'Active',
            'is_current': True,
            'license_description': 'Dispensary',
            'us_state': 'MO',
            'expiration_date': None,
            'license_category': 'Retailer'
            },
        ],
    },
            headers={'x-api-key': 'ASYNC-SERVER-API-KEY-6Lfy84qn9ew93ZwMXSYhdwqj'}
#             {'x-api-key': '80030179832729980933'}
        )

In [ ]:
new_main[new_main['license_number'] == 'DIS000091']

In [ ]:
print(stop)

#### In Case Double Checking is Needed

In [ ]:
in_metabase = pd.read_json('query_result_2021-09-27T19_06_51.847Z.json')

In [ ]:
missing_list = []

for lic in new_main['license_number'].unique():
    if lic not in list(in_metabase['License Number'].unique()):
        missing_list.append(lic)

In [ ]:
new_main = new_main[new_main['license_number'].isin(missing_list)]

In [ ]:
new_main